In [1]:
%%capture
!pip install datasets

In [2]:
!rm -rf Conditional-Variational-Transformer/
!git clone https://github.com/safakkbilici/Conditional-Variational-Transformer
!pwd

Cloning into 'Conditional-Variational-Transformer'...
remote: Enumerating objects: 427, done.
remote: Counting objects: 100% (427/427), done.
remote: Compressing objects: 100% (311/311), done.
remote: Total 427 (delta 237), reused 267 (delta 103), pack-reused 0
Receiving objects: 100% (427/427), 25.33 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (237/237), done.
/content


In [ ]:
cd Conditional-Variational-Transformer/

In [10]:
import torch
import pandas as pd
from tqdm.auto import tqdm
import json
from tokenizer.basic_tokenizer import BasicTokenizer
from models.variational_transformer import CVAETransformer
from utils.model_utils import sample_from_prior
from models.masking import get_subsequent_mask
import torch.nn.functional as F
import pandas as pd

In [5]:
tokenizer = BasicTokenizer()
tokenizer.load("/content/drive/MyDrive/nova/TCVAE/generate")

with open(f'/content/drive/MyDrive/nova/TCVAE/generate/model_params.json', 'r') as fp:
    model_params = json.load(fp)

model_params["n_src_vocab"] = 63055
model_params["n_trg_vocab"] = 63055
device = torch.device("cuda")
model = CVAETransformer(**model_params).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/nova/TCVAE/generate/model.pt"))

<All keys matched successfully>

In [ ]:
num_classes = 2
num_samples = 2500
n_len = 100
total = num_classes * num_samples * (n_len - 2)
generated = {str(i): [] for i in range(num_classes)}
with tqdm(total = total, leave=False, desc='Generation round', position = 0) as gen:
    for i in range(num_classes):
        for j in range(num_samples):
            prior = sample_from_prior(1, model_params["latent_size"], device)
            prior[:,0] = i
            trg = [tokenizer.start_token_id]
            trg_seq = torch.Tensor(trg)[None, :].long().to(device)
            with torch.no_grad():
                for step in range(2, n_len):
                    #model.eval()
                    trg_mask = get_subsequent_mask(trg_seq)
                    dec_output = model.generate(trg_seq, trg_mask, prior, None)
                    gen_seq = F.softmax(model.trg_word_prj(dec_output), dim=-1).squeeze(0).squeeze(0)
                    try:
                        max_prob = gen_seq[-1,:].argmax(dim = -1)
                    except:
                        max_prob = gen_seq.argmax(dim = -1)
                    trg.append(max_prob.item())
                    trg_seq = torch.Tensor(trg)[None, :].long().to(device)
                    gen.update()
            generated[str(i)].append(' '.join(tokenizer.decode(trg, remove_special_tokens=False)))

Generation round:   0%|          | 0/490000 [00:00<?, ?it/s]

In [13]:
def clean(sentence):
    special_chars = ["[START]", "[END]"]
    sent = sentence.split()
    for token in special_chars:
        sent = list(filter((token).__ne__, sent))
    sent = ' '.join(sent)
    return sent

In [14]:
generated_df = pd.DataFrame({"0": generated["0"], "1": generated["1"]})
generated_df["0"] = generated_df["0"].apply(clean)
generated_df["1"] = generated_df["1"].apply(clean)

In [27]:
pd.set_option('display.max_columns', 2)
pd.set_option('display.width', 10000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

In [29]:
generated_df.head(10)

,0,1
0,This movie is an embarrassment to being made it There are a religious joke or a few things that was given it a Stupid storyline and there are all over the same themes in this film If you are not laughing it you may think that this is a horrible movie I think this one is a very worst movie ever made I think anything has a very bad movie The acting is ridiculous 4 and the performance is far better than the cast is atrocious I believe that others who would be in the first time,I had to see this movie and although I didnt enjoy it If you can think of the best part of the movie takes you once in the second episode when you will never see the movie or something else to come close to creating a mystery and imagination can be a new cultural love story about heaven and the film is that we have to know the story about friendship They are being the lover of the great red herring as we did not know much compassion out loud This is the part of the character
1,I watched this movie and I thought it was a theatrical release It was mildly entertaining and effective To me the audience was on the storyline The characters arent all around especially considering how they couldnt relate to the characters and make sense of the plot lines that make no sense The characters really interesting plot was just dreadful the plot was just ridiculous The plot was also pathetic The acting was interesting and the directing the editing the actual music was not all the characters and the special effects were practically all the roles of the,This is the second film in the series the nature of the father the Europe of a slow moving but also quickly otherwise a very nice girl who gets attacked by the end of a psychiatrist who has suddenly vanished for no apparent reason Her search leads to a chance for no reason The murders are in style and respect they succeeds at as in the film is hardly in a film In the first ten minutes its appeal All the film is as well as the appeal to the way we see films and the problems
2,This movie is a good story but it is a good movie The artistic setting are restrained and a brilliant script What was a typical believable Every scene is pretty realistic and a little bit of tension Which is not to be a comedy or if you can tell you the story is taken away from the characters and the ending Think shocking Just like Just that the story was taken away from an disappointment It was a little nudity and the final scene at a quick point plus in this movie the beginning of the movie,I was a lot of the actors and I love this movie Its very curious to see how much longer The reason is the film is probably the best thing about the film is that it has made the characters in this movie The reason I watched it was made and the ending in the first time I believe that there was absolutely no definite must be made and feel for people to watch this movie The movie is very good I would recommend this movie to all comedies people who are SO far away from the
3,This movie is a real torture fest When I saw it first saw the trailer for this movie was made by some time The violence and special effects are lame Basically the movie is that the only is the worst film I ever seen I have seen the box of the movie is terrible The sound and the film reminded me of my comment is that the old wondering if you sat there to watch this movie I would be a serious movie with a poor special effects and the purpose of making this movie If,I was a kid when I was adopted and had been older I saw it on DVD I was so much older I loved it and loved it Then again after the first time I finally got the least one I was fully disappointed when it came out I was a kid back then I loved it So I was hooked and when I saw it Then when the first time I watched it back when I first saw it in the back I was hooked on the back I had two parents Then the channel will
4,This movie is a weak script and bad acting is contrived It is just an interesting story It is just a ridiculous story and